### References

* Chatbot with TF: https://chatbotsmagazine.com/contextual-chat-bots-with-tensorflow-4391749d0077
* Good reference list for reading: http://www.wildml.com/2016/04/deep-learning-for-chatbots-part-1-introduction/


### Options:
* Use categories from semtype tagging to identify

## Initialize

In [1]:
# Utility
import sys,os
import time

import pandas as pd
pd.set_option('display.max_columns', 500) # more columns displayed at once
pd.options.display.max_colwidth = 200 # more of the text displayed at once

# Custom
from processing import tag_utterances
from processing import load_sem_types

## NLP
import spacy

# Set absolute path the QuickUMLS Server
abs_path_umls = '/Users/austinpowell/Google_Drive/kp_datascience/doctor_notes/ontology/UMLS'
abs_path_data_umls = '/Users/austinpowell/Google_Drive/kp_datascience/doctor_notes/ontology/UMLS/QuickUMLS_db'
sys.path.append(abs_path_umls+'/QuickUMLS')
from quickumls import QuickUMLS
tagger = QuickUMLS(abs_path_data_umls)

In [56]:
path_to_data = '../data/reddit_comments_askDocs_2014_to_2018_03.gz'
df = pd.read_csv(path_to_data,low_memory=False)
print('Shape',df.shape)
df.head(2)

Shape (557648, 21)


,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason
0,for a manlet such as yourself I'd recommend at...,NaN,NaN,NaN,-Ai,This user has not yet been verified.,NaN,1513411674,t5_2xtuc,t3_7k5x2h,t3_7k5x2h,0,1.514772e+09,0,0,drbt2db,AskDocs,NaN,NaN,default,NaN
1,Thank you very much for answering!,NaN,NaN,NaN,-SY,This user has not yet been verified.,NaN,1445798103,t5_2xtuc,t3_3q697b,t1_cwcf958,2,1.447190e+09,0,0,cwcfjpr,AskDocs,2.0,NaN,default,NaN


In [69]:
print(len(df['link_id'].unique())/df.shape[0])
print(len(df['id'].unique())/df.shape[0])
print(len(df['parent_id'].unique())/df.shape[0])


0.2661284537916392
0.9484190743981866
0.7033110492640519


## Data Dictionary:
Original post can be seen with the link_id
* **link_id** Link to the page where the original thread was started
    * **id**
    * **parent_id** These ids are pointing back towards the original post and represent a new comment that is replying directly towards the original posting. Can be considered a general comment. I.e. All posts with the same parent_id (following the "_") as the link_id are. E.g. If the link_id is "t3_827pgt", all parent_id's with "827pgt" are pointing towards that original post.

* **subreddit_id** and **subreddit** Irrelevant

In [ ]:
# rename id columns to be more intuitive for a post

In [98]:
pdf.loc[df.parent_id.str.contains('827pgt',na=False)]rint(df.loc[df.link_id.str.contains('t3_827pgt',na=False)].shape)
print(df.loc[df.link_id.str.contains('827pgt',na=False)].shape)

print(.shape)

(44, 22)
(44, 22)
(11, 22)


In [120]:
df.loc[df.link_id.str.contains('827pgt',na=False)].shape

(44, 22)

In [107]:
df.loc[df.parent_id.str.contains('827pgt',na=False)]

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,pd
47080,"I'm not a doctor, but do you guys think this is Ankylosing Spondylitis?",NaN,NaN,NaN,Yossi25,This user has not yet been verified.,NaN,1522308033,t5_2xtuc,t3_827pgt,t3_827pgt,1,1.525707e+09,0,0,dwgoznj,AskDocs,NaN,NaN,default,NaN,827pgt
59878,"Degenerative issues run in my family. From a young age, I was showing signs of DDD and it has progressed as the years go on. I wouldn't call my case severe, but enough to where I had spent a solid...",NaN,NaN,NaN,vanteal,This user has not yet been verified.,NaN,1520299064,t5_2xtuc,t3_827pgt,t3_827pgt,5,1.524857e+09,0,0,dv8rpap,AskDocs,NaN,NaN,default,NaN,827pgt
154535,[deleted],NaN,NaN,NaN,[deleted],NaN,NaN,1520316050,t5_2xtuc,t3_827pgt,t3_827pgt,1,1.524865e+09,0,0,dv95z1b,AskDocs,NaN,NaN,NaN,NaN,827pgt
189543,Im so sorry for your loss. I’m not a doctor so please don’t take this as fact as it’s based on my personal opinion and experience. My last MRI was pretty similar and I’ve been in pain 24 hours a d...,NaN,NaN,NaN,twinkie45,This user has not yet been verified.,NaN,1520287643,t5_2xtuc,t3_827pgt,t3_827pgt,6,1.524851e+09,0,0,dv8go5a,AskDocs,NaN,NaN,default,NaN,827pgt
195752,"Sorry for your loss. It is never easy, she had a reason.\nStill leaves a lot of pain behind.\n\nI have degeneration in my neck but it is nothing like what your mother may have had. As I recall the...",NaN,NaN,NaN,DennyBenny,This user has not yet been verified.,NaN,1520297740,t5_2xtuc,t3_827pgt,t3_827pgt,2,1.524856e+09,0,0,dv8qg82,AskDocs,NaN,NaN,default,NaN,827pgt
253902,"It's not remotely the same but my father was about the same age as your mother, and in the end he chose not to go back to the hospital to treat his COPD, choosing to die at the seniors centre he w...",NaN,NaN,NaN,hobbitlover,NaN,NaN,1520298169,t5_2xtuc,t3_827pgt,t3_827pgt,6,1.524856e+09,0,0,dv8que6,AskDocs,NaN,NaN,NaN,NaN,827pgt
254645,She had degenerative disc and thinning disc from poor posture and old age. See pic: https://www-epainassist-com.cdn.ampproject.org/ii/w1000/s/www.epainassist.com/images/can-you-get-degenerative-di...,NaN,NaN,NaN,jbjbjb55555,NaN,NaN,1520310309,t5_2xtuc,t3_827pgt,t3_827pgt,-2,1.524862e+09,1,0,dv91xtu,AskDocs,NaN,NaN,NaN,NaN,827pgt
348588,"Her fears of further degeneration and living in pain, would be My guess. Im sorry for your loss.",NaN,NaN,NaN,MDFrankensteen,This user has not yet been verified.,NaN,1520274122,t5_2xtuc,t3_827pgt,t3_827pgt,45,1.524842e+09,0,0,dv81ye0,AskDocs,NaN,NaN,default,NaN,827pgt
390103,that looks alot like my MRI. I also have ddd. It's pretty painful and at times I have thought I didn't want to continue all the way out to the end of the disease because my pain is usually at leas...,NaN,NaN,NaN,molotavcocktail,This user has not yet been verified.,NaN,1520279487,t5_2xtuc,t3_827pgt,t3_827pgt,18,1.524845e+09,0,0,dv87o36,AskDocs,NaN,NaN,default,NaN,827pgt
405258,So sorry for your loss.\n\nRoughly read through the report and from what I'm able to interpret it seems like your Mother is suffering from severe degeneration of the spine which resulted in multip...,NaN,NaN,NaN,Sgmedicalstudent,NaN,NaN,1520274423,t5_2xtuc,t3_827pgt,t3_827pgt,81,1.524842e+09,0,0,dv829t2,AskDocs,NaN,NaN,NaN,NaN,827pgt


In [110]:
df.loc[df.parent_id.str.contains('dv8rpap',na=False)]

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,pd
223990,"Wow, thank you for your story. I do not believe my mother was a coward in the slightest. She was a very strong woman and she did what she had to do. I’m not struggling against the grief, I’m float...",NaN,NaN,NaN,kiwikellie,NaN,NaN,1520302687,t5_2xtuc,t3_827pgt,t1_dv8rpap,5,1.524859e+09,0,0,dv8v4ny,AskDocs,NaN,NaN,NaN,NaN,dv8rpap


In [111]:
df.loc[df.parent_id.str.contains('dv8v4ny',na=False)]

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,pd
226939,"I can’t help with your particular question, but I just wanted to say that you seem like a very compassionate, kind, and strong person, and I really wish you the best. I’m so sorry for your loss.",NaN,NaN,NaN,pegmatitic,This user has not yet been verified.,NaN,1520306013,t5_2xtuc,t3_827pgt,t1_dv8v4ny,5,1.524860e+09,0,0,dv8y923,AskDocs,NaN,NaN,default,NaN,dv8v4ny


In [90]:
df.loc[df.parent_id.str.contains('t1_dv81ye0',na=False)].loc[df['author']=='kiwikellie']

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,pd
223992,Thank you very much. I'm not wondering why she did what she did. I absolutely understand why. I'm just looking for answers on what was wrong with her spine.,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520274235,t5_2xtuc,t3_827pgt,t1_dv81ye0,9,1.524842e+09,0,0,dv822m9,AskDocs,NaN,NaN,NaN,NaN,dv81ye0


In [81]:
df.loc[df.link_id.str.contains('t3_827pgt',na=False)]

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,pd
223992,Thank you very much. I'm not wondering why she did what she did. I absolutely understand why. I'm just looking for answers on what was wrong with her spine.,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520274235,t5_2xtuc,t3_827pgt,t1_dv81ye0,9,1.524842e+09,0,0,dv822m9,AskDocs,NaN,NaN,NaN,NaN,dv81ye0
348514,Degenerative disc disease.,NaN,NaN,NaN,MDFrankensteen,This user has not yet been verified.,NaN,1520274771,t5_2xtuc,t3_827pgt,t1_dv822m9,10,1.524842e+09,0,0,dv82n3m,AskDocs,NaN,NaN,default,NaN,dv822m9
121124,"Reading through and I’m not a specialist in this field but patients I’ve had with similar impressions:\n\nNo matter what could’ve been done here she would be living with some degree of pain, which...",NaN,NaN,NaN,HokayeZeZ,This user has not yet been verified.,NaN,1520285562,t5_2xtuc,t3_827pgt,t1_dv829t2,18,1.524849e+09,0,0,dv8ed1z,AskDocs,NaN,NaN,default,NaN,dv829t2
223985,Thank you. I believe she was told she would eventually become completely paralyzed. Do you believe that is something this condition could eventually progress to?,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520274604,t5_2xtuc,t3_827pgt,t1_dv829t2,35,1.524842e+09,0,0,dv82gtb,AskDocs,NaN,NaN,NaN,NaN,dv829t2
42816,"Yes, unfortunately. It seems she had [myelomalacia](https://en.wikipedia.org/wiki/Myelomalacia) at the level around the third cervical vertebra (C3), which would eventually lead to paralysis of ev...",NaN,NaN,NaN,Phhhhuh,This user has not yet been verified.,NaN,1520357456,t5_2xtuc,t3_827pgt,t1_dv82gtb,3,1.524880e+09,0,0,dv9x79p,AskDocs,NaN,NaN,default,NaN,dv82gtb
78045,"If left untreated, yes.",NaN,NaN,NaN,Pinky135,"B.S., Medical Lab Sciences",NaN,1520275503,t5_2xtuc,t3_827pgt,t1_dv82gtb,36,1.524843e+09,0,0,dv83f4u,AskDocs,NaN,NaN,verified-mid,NaN,dv82gtb
7937,"Which isnt a disease, its a misnomer. We all have DDD but some worse than others.",NaN,NaN,NaN,Jae_t,This user has not yet been verified.,NaN,1520286469,t5_2xtuc,t3_827pgt,t1_dv82n3m,-5,1.524850e+09,1,0,dv8fd0y,AskDocs,NaN,NaN,default,NaN,dv82n3m
223997,What would the treatment options have been for her? Would surgery have been out of the question due to her age?,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520276053,t5_2xtuc,t3_827pgt,t1_dv83f4u,20,1.524843e+09,0,0,dv840aq,AskDocs,NaN,NaN,NaN,NaN,dv83f4u
78071,"I can't answer that. It depends on a lot of factors and only the treating doctor can give a treatment plan, if one is possible.",NaN,NaN,NaN,Pinky135,"B.S., Medical Lab Sciences",NaN,1520276404,t5_2xtuc,t3_827pgt,t1_dv840aq,4,1.524843e+09,0,0,dv84dtd,AskDocs,NaN,NaN,verified-mid,NaN,dv840aq
273301,"if left untreated it wouldve in ended in complete paralysis, only possible treatment is surgery, in a very high risk area, which might’ve ended up paralyzing her.\n\nI am so sorry for your loss",NaN,NaN,NaN,L_AlAbdallat,This user has not yet been verified.,NaN,1520278032,t5_2xtuc,t3_827pgt,t1_dv840aq,33,1.524844e+09,0,0,dv8643f,AskDocs,NaN,NaN,default,NaN,dv840aq


In [83]:
df.loc[df['author']=='kiwikellie']

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,pd
223985,Thank you. I believe she was told she would eventually become completely paralyzed. Do you believe that is something this condition could eventually progress to?,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520274604,t5_2xtuc,t3_827pgt,t1_dv829t2,35,1.524842e+09,0,0,dv82gtb,AskDocs,NaN,NaN,NaN,NaN,dv829t2
223986,Thank you. It definitely helps to learn as much about her condition as I can.,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520278118,t5_2xtuc,t3_827pgt,t1_dv8643f,14,1.524844e+09,0,0,dv867dt,AskDocs,NaN,NaN,NaN,NaN,dv8643f
223987,"Thank you so much. We all do the best we can with what was handed to us. All the best for you, as well.",NaN,NaN,NaN,kiwikellie,NaN,NaN,1520307281,t5_2xtuc,t3_827pgt,t1_dv8y923,3,1.524861e+09,0,0,dv8zdsb,AskDocs,NaN,NaN,NaN,NaN,dv8y923
223988,Can you please elaborate?,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520288550,t5_2xtuc,t3_827pgt,t1_dv8fd0y,3,1.524851e+09,0,0,dv8hmg0,AskDocs,NaN,NaN,NaN,NaN,dv8fd0y
223989,I think she was already experiencing such pain and stiffness that the thought that it would get worse and worse was too overwhelming for her to bear. I think she made the best choice for herself.\...,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520288433,t5_2xtuc,t3_827pgt,t1_dv8ed1z,8,1.524851e+09,0,0,dv8hi3n,AskDocs,NaN,NaN,NaN,NaN,dv8ed1z
223990,"Wow, thank you for your story. I do not believe my mother was a coward in the slightest. She was a very strong woman and she did what she had to do. I’m not struggling against the grief, I’m float...",NaN,NaN,NaN,kiwikellie,NaN,NaN,1520302687,t5_2xtuc,t3_827pgt,t1_dv8rpap,5,1.524859e+09,0,0,dv8v4ny,AskDocs,NaN,NaN,NaN,NaN,dv8rpap
223991,"I’m glad to hear that she might have had another option, but also sad that I couldn’t convince her to talk to another doctor. She was so sure that there was nothing that could be done for her.\n\n...",NaN,NaN,NaN,kiwikellie,NaN,NaN,1520287993,t5_2xtuc,t3_827pgt,t1_dv8fe0l,12,1.524851e+09,0,0,dv8h1e9,AskDocs,NaN,NaN,NaN,NaN,dv8fe0l
223992,Thank you very much. I'm not wondering why she did what she did. I absolutely understand why. I'm just looking for answers on what was wrong with her spine.,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520274235,t5_2xtuc,t3_827pgt,t1_dv81ye0,9,1.524842e+09,0,0,dv822m9,AskDocs,NaN,NaN,NaN,NaN,dv81ye0
223993,Thank you for a more detailed explanation.\n\nI’m not upset at my mother for making the choice she did and I don’t accuse her of making a bad choice. She cared for my father through his cancer all...,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520281114,t5_2xtuc,t3_827pgt,t1_dv87o36,24,1.524846e+09,0,0,dv89g6u,AskDocs,NaN,NaN,NaN,NaN,dv87o36
223994,Thank you for such an in-depth explanation. I really appreciate you taking the time to write this for me.\n\nI lost my father when I was 9 and now my mother at 29. I grew up understanding that my ...,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520313546,t5_2xtuc,t3_827pgt,t1_dv930hi,13,1.524864e+09,0,0,dv94al4,AskDocs,NaN,NaN,NaN,NaN,dv930hi


In [71]:
df.loc[df.pd.str.contains('827pgt',na=False)]

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,pd
47080,"I'm not a doctor, but do you guys think this i...",NaN,NaN,NaN,Yossi25,This user has not yet been verified.,NaN,1522308033,t5_2xtuc,t3_827pgt,t3_827pgt,1,1.525707e+09,0,0,dwgoznj,AskDocs,NaN,NaN,default,NaN,827pgt
59878,Degenerative issues run in my family. From a y...,NaN,NaN,NaN,vanteal,This user has not yet been verified.,NaN,1520299064,t5_2xtuc,t3_827pgt,t3_827pgt,5,1.524857e+09,0,0,dv8rpap,AskDocs,NaN,NaN,default,NaN,827pgt
154535,[deleted],NaN,NaN,NaN,[deleted],NaN,NaN,1520316050,t5_2xtuc,t3_827pgt,t3_827pgt,1,1.524865e+09,0,0,dv95z1b,AskDocs,NaN,NaN,NaN,NaN,827pgt
189543,Im so sorry for your loss. I’m not a doctor so...,NaN,NaN,NaN,twinkie45,This user has not yet been verified.,NaN,1520287643,t5_2xtuc,t3_827pgt,t3_827pgt,6,1.524851e+09,0,0,dv8go5a,AskDocs,NaN,NaN,default,NaN,827pgt
195752,"Sorry for your loss. It is never easy, she had...",NaN,NaN,NaN,DennyBenny,This user has not yet been verified.,NaN,1520297740,t5_2xtuc,t3_827pgt,t3_827pgt,2,1.524856e+09,0,0,dv8qg82,AskDocs,NaN,NaN,default,NaN,827pgt
253902,It's not remotely the same but my father was a...,NaN,NaN,NaN,hobbitlover,NaN,NaN,1520298169,t5_2xtuc,t3_827pgt,t3_827pgt,6,1.524856e+09,0,0,dv8que6,AskDocs,NaN,NaN,NaN,NaN,827pgt
254645,She had degenerative disc and thinning disc fr...,NaN,NaN,NaN,jbjbjb55555,NaN,NaN,1520310309,t5_2xtuc,t3_827pgt,t3_827pgt,-2,1.524862e+09,1,0,dv91xtu,AskDocs,NaN,NaN,NaN,NaN,827pgt
348588,Her fears of further degeneration and living i...,NaN,NaN,NaN,MDFrankensteen,This user has not yet been verified.,NaN,1520274122,t5_2xtuc,t3_827pgt,t3_827pgt,45,1.524842e+09,0,0,dv81ye0,AskDocs,NaN,NaN,default,NaN,827pgt
390103,that looks alot like my MRI. I also have ddd. ...,NaN,NaN,NaN,molotavcocktail,This user has not yet been verified.,NaN,1520279487,t5_2xtuc,t3_827pgt,t3_827pgt,18,1.524845e+09,0,0,dv87o36,AskDocs,NaN,NaN,default,NaN,827pgt
405258,So sorry for your loss.\n\nRoughly read throug...,NaN,NaN,NaN,Sgmedicalstudent,NaN,NaN,1520274423,t5_2xtuc,t3_827pgt,t3_827pgt,81,1.524842e+09,0,0,dv829t2,AskDocs,NaN,NaN,NaN,NaN,827pgt


In [3]:
# Load semantic types
sem_type_dict = load_sem_types('../data/SemGroups_2013.txt')

In [4]:
t = tagger.nlp(df['body'].iloc[4])
t.text

"She's had it for 8 months, we've never had any issues with it before."

In [7]:
t = tagger.nlp(t.text)

s = t
matches= tagger.match(s, best_match=True, ignore_syntax=False)
for match in matches:
    dir(match)
    print(match,'\n')
for m in match:
    print(m)

[{'start': 19, 'end': 25, 'ngram': 'months', 'term': 'month', 'cui': 'C1561542', 'similarity': 0.75, 'semtypes': {'T170'}, 'preferred': 1}] 

[{'start': 47, 'end': 53, 'ngram': 'issues', 'term': 'issue', 'cui': 'C0033213', 'similarity': 0.75, 'semtypes': {'T033'}, 'preferred': 1}, {'start': 47, 'end': 53, 'ngram': 'issues', 'term': 'issue', 'cui': 'C1706387', 'similarity': 0.75, 'semtypes': {'T170'}, 'preferred': 1}] 

{'start': 47, 'end': 53, 'ngram': 'issues', 'term': 'issue', 'cui': 'C0033213', 'similarity': 0.75, 'semtypes': {'T033'}, 'preferred': 1}
{'start': 47, 'end': 53, 'ngram': 'issues', 'term': 'issue', 'cui': 'C1706387', 'similarity': 0.75, 'semtypes': {'T170'}, 'preferred': 1}


In [21]:
sd = load_sem_types('../data/SemGroups_2013.txt')
m['semtypes']

AttributeError: 'set' object has no attribute 'text'

In [11]:
import importlib
from processing import tag_utterances
from processing import load_sem_types

tag_utterances(1, t.text, tagger)

{'T170'}


[[1, 19, 25, 'month', 'C1561542', 0.75, {'Intellectual Product'}]]

In [8]:
df.shape

(557648, 21)

In [9]:
%%time
print('Iterating over every document')
#Iterate over every document and extract the concepts
i=-1        
result = []
for idx,doc in  enumerate(df['body']):

    if idx % 10000 == 0:
        print("Documents processed: {}".format(idx))
    try:
        i+=1
        annotations = tag_utterances(i,doc,tagger)
        result.extend(annotations)
    except Exception as e:
        print(e)
        
df_matches = pd.DataFrame(data=result, columns =['document','start','end','term','cui','similarity','semtypes'])
df_matches.sort_values(by=['document','start'],inplace=True)

Iterating over every document
Documents processed: 0
Documents processed: 10000
Documents processed: 20000
Documents processed: 30000
Documents processed: 40000
Documents processed: 50000
Argument 'string' has incorrect type (expected str, got float)
Documents processed: 60000
Documents processed: 70000
Argument 'string' has incorrect type (expected str, got float)
Documents processed: 80000
Documents processed: 90000
Documents processed: 100000
Documents processed: 110000
Documents processed: 120000
Argument 'string' has incorrect type (expected str, got float)
Documents processed: 130000
Documents processed: 140000
Documents processed: 150000
Documents processed: 160000
Documents processed: 170000
Documents processed: 180000
Documents processed: 190000
Argument 'string' has incorrect type (expected str, got float)
Documents processed: 200000
Argument 'string' has incorrect type (expected str, got float)
Argument 'string' has incorrect type (expected str, got float)
Documents processe

KeyboardInterrupt: 

In [6]:
df.head()

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,...,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason
0,for a manlet such as yourself I'd recommend at...,NaN,NaN,NaN,-Ai,This user has not yet been verified.,NaN,1513411674,t5_2xtuc,t3_7k5x2h,...,0,1.514772e+09,0,0,drbt2db,AskDocs,NaN,NaN,default,NaN
1,Thank you very much for answering!,NaN,NaN,NaN,-SY,This user has not yet been verified.,NaN,1445798103,t5_2xtuc,t3_3q697b,...,2,1.447190e+09,0,0,cwcfjpr,AskDocs,2.0,NaN,default,NaN
2,Never been tested for that. I was hoping the ...,NaN,NaN,NaN,-o2,This user has not yet been verified.,NaN,1461952470,t5_2xtuc,t3_4gz1fi,...,1,1.463777e+09,0,0,d2mce34,AskDocs,1.0,NaN,default,NaN
3,"She said her constant abdominal pain is a 6, t...",NaN,NaN,NaN,05P,This user has not yet been verified.,NaN,1504214332,t5_2xtuc,t3_6x9jk0,...,1,1.504553e+09,0,0,dme9lzr,AskDocs,NaN,NaN,default,NaN
4,"She's had it for 8 months, we've never had any...",NaN,NaN,NaN,05P,This user has not yet been verified.,NaN,1504217835,t5_2xtuc,t3_6x9jk0,...,1,1.504554e+09,0,0,dmecohs,AskDocs,NaN,NaN,default,NaN


In [9]:
df.author_flair_text.value_counts()

This user has not yet been verified.     352466
Physician                                 26484
Medical Student                           13399
B.S., Medical Lab Sciences                 3069
FY1 Doctor                                 2860
Moderator                                  2644
Registered Nurse                           2589
Emergency Medical Technician               2023
Surgeon                                    1765
Surgeon | Moderator                        1710
Orthopaedic Surgeon                        1569
Psychiatrist                               1513
Dermatologist                              1481
Doctor (A&amp;E)                           1435
Pharm.D. - Hospital Pharmacist             1056
Emergency Physician                        1027
Nursing Graduate, RPN                       803
Pharm.D.                                    748
Nursing Student                             735
Occupational Therapist                      727
Anesthesiologist                        

### Predicting Clinician

http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import time

In [5]:
df['is_clinician'] = df['author_flair_text'].apply(lambda r: 0 if r =='This user has not yet been verified.' else 1)
print(df['is_clinician'].mean())
print(df['is_clinician'].value_counts())

0.36794178406449946
0    352466
1    205182
Name: is_clinician, dtype: int64


In [6]:
import pandas as pd
import nltk

df['tokenized_sents'] = df['body'].apply(lambda row: str(row).strip().replace('\n','').lower().split(' '))

In [34]:
df['body']  = df['body'].apply(lambda r: str(r))
df['body'].dtypes

dtype('O')

In [36]:
df['tokenized_sents'] = df['body'].apply(lambda row: nltk.word_tokenize(row))

KeyboardInterrupt: 

In [30]:
X = df['tokenized_sents'].tolist()
# Remove empty strings
X = [[t for t in sent if len(t)>1] for sent in X]

In [22]:
import numpy as np

with open("../data/glove.6B.50d.txt", "rb") as lines:
    w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

In [23]:
import gensim
# let X be a list of tokenized texts (i.e. list of lists of tokens)
model = gensim.models.Word2Vec(X, size=100)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/Users/austinpowell/Google_Drive/kp_datascience/virtual_envs/nlp-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.


We got ourselves a dictionary mapping word -> 100-dimensional vector. Now we can use it to build features. The simplest way to do that is by averaging word vectors for all words in a text. We will build a sklearn-compatible transformer that is initialised with a word -> vector dictionary.

In [9]:
class MeanEmbeddingVectorizer(object):
    """
    Average word vectors for all words in a text using n dimensional word embedding vector.
    """
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(word2vec.values())

    def fit(self, X, y):
        return self

    def transform(self, X):
        # If a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    # If text is empty, return vector of zeros
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

Let’s throw in a version that uses tf-idf weighting scheme for good measurem

In [10]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.values())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

etree_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200,n_jobs=7))])

etree_w2v_tfidf = Pipeline([
    ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200,n_jobs=7))])

NameError: name 'w2v' is not defined

In [28]:
%%time
etree_w2v.fit(X,df['is_clinician'])

ValueError: setting an array element with a sequence.

**Bag of Words**

In [32]:
# Create pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('vect', CountVectorizer(lowercase=False,ngram_range=(1,2))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())
                    ])

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,df['is_clinician'] , test_size=0.2, random_state=329)

In [34]:
%%time
from sklearn.model_selection import cross_val_score

scores = cross_val_score(text_clf, X_train, y_train, cv=5,n_jobs=7,scoring='f1_macro')

JoblibTypeError: JoblibTypeError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Lib

In [19]:
print("F1 macro: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

NameError: name 'scores' is not defined